In [110]:
# Day 5 challenge for AOC2019
# https://adventofcode.com/2019/day/5

In [123]:
# this object is where the Intcode program lives and some useful methods
# to fill it, clear it, read from it, write to it
class Memory:
    global debug
    def __init__(self, size):
        self.size = size  # must specify the memory size
        self.storage = [None] * size  # initialize with None
     
    # dump out the contents between 2 positions (defaults all the memory)
    def dump(self,start=0,end=-1):
        if(end == -1):
            end = self.size
        for i in range(start, end):
            print("{}: {}".format(i,self.storage[i]))

    # fill the memory with a sequence, starting at a certain position - which defaults to 0
    def fill(self, codelist, start_pos = 0):
        i = start_pos
        for j in codelist:
            self.storage[i] = j
            i+=1
    
    # fill (write to) memory with a sequence, allowing direct or reference starting location
    def mode_fill(self, s, mem_location, mode):
        # mem_location: where to begin fill or to reference to begin
        # mode (0 - position, 1 - immediate)
        if(debug):
            print(".mode_fill(s: {}, mem_location: {}, mode: {})".format(s, mem_location, mode))
        if(mode == '0'):
            i = self.storage[mem_location]
            for j in s:
                if(debug):
                    print("..filling \{}\ into pos:{}".format(j, i))
                self.storage[i] = j
                i+=1
        elif(mode == '1'):
            i = mem_location
            for j in s:
                self.storage[i] = j
                i+=1

    # read the contents from just one position
    def read(self, pos):
        return self.storage[pos]
   
    # return the value based on parameter mode (position or immediate)
    def mode_read(self, pos, mode):
        # mode: 0 - position, 1 - immediate
        if(debug):
            print("mode_read(pos:{}, mode:{})".format(pos, mode))
        if(mode == 0):
            return self.storage[self.storage[pos]]
        elif(mode == 1):
            return self.storage[pos]

    # clear the memory
    def clear(self):
        self.storage = [None] * self.size

In [124]:
# this function will execute a single sequence (operator + subsequent operands and output location)
def exec_seq(ctr, m):
    global debug
    if(debug):
        print("executing sequence, ctr: {}".format(ctr))
    # ctr - where to start
    # m - the memory object to work from
    # the first in the sequence is the 'enhnanced opcode' (includes 3 parameter modes in most significant position)
    # zero-pad to 5 - most significant 0 is implied
    enhanced_opcode = str(m.read(ctr)).zfill(5)
    # first 3 characters are the parameter modes
    pmodes = enhanced_opcode[:3]
    # last 2 characters are the opcode
    opcode = enhanced_opcode[3:]
    # if it is 99, we're done
    if(debug):
        print("opcode: {}".format(opcode))
        print("pmodes: {}".format(pmodes))
    if(opcode == 99):
        return("STOP")
    # this is the "add" operator
    elif(opcode == "01"):
        operand1 = m.mode_read(ctr+1, pmodes[2])
        operand2 = m.mode_read(ctr+2, pmodes[1])
        calcval = operand1 + operand2
    # this is the "multiply" operator
    elif(opcode == "02"):
        operand1 = m.mode_read(ctr+1, pmodes[2])
        operand2 = m.mode_read(ctr+2, pmodes[1])
        calcval = operand1 * operand2
    # this is the 'store from input' operator
    elif(opcode == "03"):
        # get the input
        val = getfrominput()
        if(debug):
            print("opcode: {}, val: {}".format(opcode, val))
        # store it in address provided by next code
        m.mode_fill(val, ctr+1, pmodes[2])
        # return next prog counter
        return(ctr+2)
    # this is the 'send to output' operator
    elif(opcode == "04"):
        # read the val (based on mode) specified from next code and send to output
        sendtooutput(m.mode_read(ctr+1, pmodes[2]))
        return(ctr+2)
    # if here, then the opcode is not known
    else:
        return("BADOPCODE")
    # fill the output destination with value depending on mode
    print("doing a calc write...{} to {}, pmode: {}".format([calcval], ctr+3, pmodes[0]))
    m.mode_fill([calcval], ctr+3, pmodes[0])
    # and return the next postion in the memory
    return(ctr+4)

In [125]:
def sendtooutput(s):
    print("OUTPUT: {}".format(s))

In [126]:
def getfrominput():
    global inputstring
    return(inputstring)

In [127]:
debug = True
# the input provided:
inputstring = "1"

# get the full programming sequence (the program)
seq = [3,225,1,225,6,6,1100,1,238,225,104,0,1102,57,23,224,101,-1311,224,224,4,224,1002,223,8,223,101,6,224,224,1,223,224,223,1102,57,67,225,102,67,150,224,1001,224,-2613,224,4,224,1002,223,8,223,101,5,224,224,1,224,223,223,2,179,213,224,1001,224,-469,224,4,224,102,8,223,223,101,7,224,224,1,223,224,223,1001,188,27,224,101,-119,224,224,4,224,1002,223,8,223,1001,224,7,224,1,223,224,223,1,184,218,224,1001,224,-155,224,4,224,1002,223,8,223,1001,224,7,224,1,224,223,223,1101,21,80,224,1001,224,-101,224,4,224,102,8,223,223,1001,224,1,224,1,224,223,223,1101,67,39,225,1101,89,68,225,101,69,35,224,1001,224,-126,224,4,224,1002,223,8,223,1001,224,1,224,1,224,223,223,1102,7,52,225,1102,18,90,225,1101,65,92,225,1002,153,78,224,101,-6942,224,224,4,224,102,8,223,223,101,6,224,224,1,223,224,223,1101,67,83,225,1102,31,65,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,1007,226,226,224,102,2,223,223,1005,224,329,1001,223,1,223,108,677,226,224,1002,223,2,223,1005,224,344,1001,223,1,223,1007,677,677,224,1002,223,2,223,1005,224,359,1001,223,1,223,1107,677,226,224,102,2,223,223,1006,224,374,1001,223,1,223,8,226,677,224,1002,223,2,223,1006,224,389,101,1,223,223,8,677,677,224,102,2,223,223,1006,224,404,1001,223,1,223,1008,226,226,224,102,2,223,223,1006,224,419,1001,223,1,223,107,677,226,224,102,2,223,223,1006,224,434,101,1,223,223,7,226,226,224,1002,223,2,223,1005,224,449,1001,223,1,223,1107,226,226,224,1002,223,2,223,1006,224,464,1001,223,1,223,1107,226,677,224,1002,223,2,223,1005,224,479,1001,223,1,223,8,677,226,224,1002,223,2,223,1006,224,494,1001,223,1,223,1108,226,677,224,1002,223,2,223,1006,224,509,101,1,223,223,1008,677,677,224,1002,223,2,223,1006,224,524,1001,223,1,223,1008,677,226,224,102,2,223,223,1006,224,539,1001,223,1,223,1108,677,677,224,102,2,223,223,1005,224,554,101,1,223,223,108,677,677,224,102,2,223,223,1006,224,569,101,1,223,223,1108,677,226,224,102,2,223,223,1005,224,584,1001,223,1,223,108,226,226,224,1002,223,2,223,1005,224,599,1001,223,1,223,1007,226,677,224,102,2,223,223,1005,224,614,1001,223,1,223,7,226,677,224,102,2,223,223,1006,224,629,1001,223,1,223,107,226,226,224,102,2,223,223,1005,224,644,101,1,223,223,7,677,226,224,102,2,223,223,1005,224,659,101,1,223,223,107,677,677,224,1002,223,2,223,1005,224,674,1001,223,1,223,4,223,99,226]
# create a computer object with twice the memory of that sequence
mycomp = Memory(len(seq) * 2)
# load the program
mycomp.fill(seq)


In [128]:
# execute the program
ctr = 0
ret = exec_seq(ctr, mycomp)
while(ret != "STOP"):
    ret = exec_seq(ret, mycomp)
print("FINAL STATE: {}".format(ret))

executing sequence, ctr: 0
opcode: 03
pmodes: 000
opcode: 03, val: 1
.mode_fill(s: 1, mem_location: 1, mode: 0)
..filling \1\ into pos:225
executing sequence, ctr: 2
opcode: 01
pmodes: 000
mode_read(pos:3, mode:0)
mode_read(pos:4, mode:0)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'